#Libraries Installation

In [24]:
!pip install transformers
!pip install datasets

#Import Libraries

In [16]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
from torchaudio.transforms import Resample

#Load Model

In [9]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Wav File

In [14]:
wav_file = "tune.wav"

#Loading and resamplaing to 16K

In [26]:
input_audio, original_sample_rate = torchaudio.load(wav_file)
resample_transform = Resample(original_sample_rate, processor.feature_extractor.sampling_rate)
input_audio_resampled = resample_transform(input_audio)

# Original Features 2D(mean), 1D (mean) features

In [30]:
# Convert audio to feature vector sequence
inputs = processor(input_audio_resampled.squeeze().numpy(), return_tensors="pt", padding="longest", sampling_rate=processor.feature_extractor.sampling_rate)

# The feature vectors can be obtained from the logits
with torch.no_grad():
    logits = model(input_values=inputs.input_values).logits
feature_vectors = logits.squeeze().numpy()
mean2_pooled_vector = feature_vectors.mean(axis=0)
mean1D_pooled_vector = mean2_pooled_vector.mean(axis=0)
print("1D Shape of mean-pooled vector:", mean1D_pooled_vector.shape)

print("1D mean-pooled vector:", mean1D_pooled_vector)

1D Shape of mean-pooled vector: (32,)
1D mean-pooled vector: [  7.1942234  -21.223545   -21.079628   -21.08843      1.2637048
   0.39762488  -0.09619333  -0.8019544   -0.83229023  -2.6736352
  -1.5765425   -0.8026756   -1.1163678   -0.7814915   -1.6988808
  -1.1038874   -1.9338542   -3.1454344   -2.5537903   -2.6415412
  -1.6972464   -2.384517    -2.6768792   -2.2874665   -2.3926866
  -4.372412    -2.4959414   -3.726026    -5.974325    -6.6739526
  -5.729926    -5.9502587 ]
